#### Importing Dependencies and Connection Variables

In [ ]:
# $ pip install --for connection to oracle db
# $ pip install pandas  --for data manipulation
# $ pip install openpyxl  --for writing to excel-->csv
# $ pip install python_dotenv   --for reading environment variables

import oracledb
import pandas as pd
import openpyxl
import os
from dotenv import load_dotenv

load_dotenv()

un = os.environ.get("USER_NAME")
pw = os.environ.get("PASSWORD")
host = os.environ.get("HOST")
port = os.environ.get("PORT")
serv = os.environ.get("SERVICE")

file_extensions = {1: ".csv", 2: ".xlsx"}

In [ ]:
def open_connection():
    conn, cursor = None, None
    try:
        conn = oracledb.connect(user=un, password=pw, dsn=(host + ":" + port + "/" + serv))
        cursor = conn.cursor()
        if conn.is_healthy():
            print(f"Connection to Oracle DB (version: {conn.version}) successful. \n")

    except Exception as e:
        print(f"Error connecting to Oracle DB: \n{e} \n")

    return conn, cursor


def close_connection(conn, cursor):
    if conn is not None:
        if conn.is_healthy():
            try:
                cursor.close()
                conn.close()
                print("Connection closed.\n")
            except Exception as e:
                print(f"Error closing connection: \n{e} \n")
        else:
            print("Not connected. \n")
    else:
        print("No connection to database established. \n")
        


#### Defining Functions to Open/Close Connection and Choose Result File Extension

In [ ]:
def open_connection():
    conn, cursor = None, None
    try:
        conn = oracledb.connect(user=un, password=pw, dsn=(host + ":" + port + "/" + serv))
        cursor = conn.cursor()
        if conn.is_healthy():
            print(f"Connection to Oracle DB (version: {conn.version}) successful. \n")

    except Exception as e:
        print(f"Error connecting to Oracle DB: \n{e} \n")

    return conn, cursor


def close_connection(conn, cursor):
    if conn is not None:
        if conn.is_healthy():
            try:
                cursor.close()
                conn.close()
                print("Connection closed.\n")
            except Exception as e:
                print(f"Error closing connection: \n{e} \n")
        else:
            print("Not connected. \n")
    else:
        print("No connection to database established. \n")


def choose_file_ext():
    max_attempts = 5
    Invalid_attempts = 0
    while Invalid_attempts < max_attempts:
        ext = input("Choose file extension: 1 for .csv or 2 for .xlsx: \n")
        if not ext.isdigit():
            print("Invalid input. Please enter a number: 1 for .csv or 2 for .xlsx: \n")
            Invalid_attempts += 1
            continue

        ext = int(ext)  # Convert the input to an integer

        if ext in file_extensions:
            print("You chose format:", file_extensions[ext])
            file_ext = file_extensions[ext]
            return file_ext
        else:
            print("Invalid selection. Please choose 1 for .csv or 2 for .xlsx \n")
            Invalid_attempts += 1

    if Invalid_attempts == max_attempts:
        print("Maximum attempts (5) reached. Exiting... \nPlease start again.")
        return None

#### Reading/Executing Queries and Saving Data to CSV

In [ ]:
def read_execute_save_to_csv():
    conn, cursor = open_connection() # connection to oracle db
    file_ext = choose_file_ext()
    if file_ext in file_extensions.values():
        if conn is not None:
            if conn.is_healthy():
                print("Connection open. Querying/Extracting data... \n")

                cur_dir = os.getcwd()
                result_folder_name = "Result_Files"
                query_folder_name = "Query_Files"
                result_folder_path = os.path.join(cur_dir, result_folder_name)  # Create the new folder path
                query_folder_path = os.path.join(cur_dir, query_folder_name)  # Create the new folder path
                os.makedirs(result_folder_path, exist_ok=True)  # Create the folder if it doesn't exist

                for file in os.listdir(query_folder_path):
                    try:
                        if os.path.isfile(os.path.join(query_folder_path, file)) and file.endswith(
                            ".sql"
                        ):
                            query_file_path = os.path.join(query_folder_path, file)
                            result_file_name = os.path.splitext(file)[0] + file_ext
                            result_file_path = os.path.join(result_folder_name, result_file_name)

                            with open(query_file_path, "r") as sql:
                                query = sql.read()  # reading query
                                if query.startswith("/*Execute with Python*/"):
                                    cursor.execute(query)  # executing query
                                    columns = [i[0] for i in cursor.description]  # getting column names
                                    data = cursor.fetchall()  # getting data

                                    df = pd.DataFrame(data, columns=columns)  # creating dataframe
                                    df.set_index(columns[0], inplace=True)  # setting index as first column

                                    # writing to csv
                                    if file_ext == ".csv":
                                        df.to_csv(result_file_path)

                                    # writing to excel
                                    if file_ext == ".xlsx":
                                        with pd.ExcelWriter(result_file_path) as writer:
                                            df.to_excel(writer)

                                    print("File status:", result_file_name, "saved successfully. \n")
                                else:
                                    print("File status:", file, "not queried. Query not starting with '/*Execute with Python*/'. \n")

                    except Exception as e:
                        print(f"Error reading/querying file: {file}: {e} \n")

                close_connection(conn, cursor)  # closing connection

            else:
                print("Error connecting to Oracle DB. \n")
        else:
            print("No Connection to database established. \n")
    else:
        print("Invalid file extension. Please choose 1 for .csv or 2 for .xlsx \n")


# calling the function
read_execute_save_to_csv()